In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = os.path.join(os.getcwd(), 'transformers')
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import logging
import sys

logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

In [3]:
import pfrl
from textrl import TextRLEnv, TextRLActor, train_agent_with_evaluation
from transformers import BlenderbotForConditionalGeneration, AutoTokenizer, AutoModelWithLMHead, pipeline

/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
checkpoint = "facebook/blenderbot-400M-distill"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = BlenderbotForConditionalGeneration.from_pretrained(checkpoint).to('cuda:0')

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help`
       for usage information.



In [5]:
model_copy = BlenderbotForConditionalGeneration.from_pretrained(checkpoint).to('cuda:0')

In [6]:
# sentiment_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")
# sentiment_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion").to('cuda:0')
sentiment_scorer = pipeline("sentiment-analysis", model="michellejieli/emotion_text_classifier", return_all_scores=True)

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/transformers/pipelines/text_classification.py:107: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,


In [7]:
# def get_emotion(text):
#     input_ids = sentiment_tokenizer.encode(text + '</s>', return_tensors='pt').to('cuda:0')

#     output = sentiment_model.generate(input_ids=input_ids,
#                 max_length=2)
    
#     dec = [sentiment_tokenizer.decode(ids) for ids in output]
#     label = dec[0]
#     return label.split(' ')[1]

# def get_emotion_score(text, emotion='anger'):
#     for dict_inner in sentiment_scorer(text)[0]:
#         if dict_inner['label'] == emotion:
#             return dict_inner['score']
        
def get_emotion_score(text, emotion='neagtive'):
    reward = 0
    for dict_inner in sentiment_scorer(text)[0]:
        if dict_inner['label'] in ['anger', 'disgust', 'sadness', 'fear']:
            reward += dict_inner['score']
    return reward

In [8]:
get_emotion_score('Yes. We are actually from the USA.')

0.0036078432167414576

In [9]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish):  # predicted will be the list of predicted token
        rewards = []
        inputs = tokenizer([input_item['input']], return_tensors="pt").to('cuda:0')
        reply_ids = model_copy.generate(**inputs)
        reply = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0]
        reply = reply.replace('</s>', '')
        reply = reply.replace('<s>', '')
        reply = reply.strip()
        base_score = get_emotion_score(reply)
        for i in predicted_list:
            if finish or len(i) > self.env_max_length:
                predicted_text = tokenizer.convert_tokens_to_string(i)
                predicted_text = predicted_text.replace('</s>', '')
                predicted_text = predicted_text.replace('<s>', '')
                predicted_text = predicted_text.strip()
                print(input_item['input'])
                print(reply)
                print(predicted_text)
                # reward = 10 * bleu_score.sentence_bleu(references=[word_tokenize(input_item['output'])], hypothesis=word_tokenize(predicted_text))
                # rewards.append(reward)
                reward = 0
                reward += (fuzz.ratio(reply, predicted_text) - 50)
                # reward += fuzz.ratio(reply, predicted_text)
                predicted_score = get_emotion_score(predicted_text)
                reward += 100 * (predicted_score - base_score)
                # reward += 100 * max(0, predicted_score - base_score)
                reward += 25 * (predicted_score - 0.5)
                # reward += 25 * predicted_score
                rewards.append(reward)
                # calculate reward score base on predicted_list
            else:
                rewards.append(0)
        return rewards

In [10]:
observation_list = []
with open('ijcnlp_dailydialog/dialogues_text.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

with open('ijcnlp_dailydialog/dialogues_emotion.txt', 'r', encoding='utf-8') as f:
    emotions = f.readlines()

for i in range(len(lines)):
    line_split = lines[i].split('__eou__')
    emotion_split = emotions[i].split(' ')
    for j in range(len(emotion_split) - 1):
        if emotion_split[j + 1] != '2': continue
        observation_list.append({
            'input': line_split[j].strip(' ')
        })

print(len(observation_list))
observation_list

291


[{'input': 'So Dick , how about getting some coffee for tonight ?'},
 {'input': 'Isn ’ t he the best instructor ? I think he ’ s so hot . Wow ! I really feel energized , don ’ t you ?'},
 {'input': 'What ’ s wrong ? Didn ’ t you think it was fun ? !'},
 {'input': 'What ?'},
 {'input': 'Cats are beautiful and lovely , aren ’ t they ?'},
 {'input': 'I don ’ t think so . I think cats are sweet .'},
 {'input': "Well . You're the tour guide , I'll follow you ."},
 {'input': 'Hi Anna , come in .'},
 {'input': 'Hey , Tom , did you enjoy your lunch ?'},
 {'input': "What ' the matter ?"},
 {'input': 'Cut it out . Look at my smile , and it seems like the flowers in the spring . Right ?'},
 {'input': 'Tom , is Jenny crying ?'},
 {'input': "I'm going to do some jogging on the treadmill . Then I think I'll do some work with the dumbbless ."},
 {'input': "Helen , why do you keep flushing the toilet ? What's wrong ?"},
 {'input': "You did what ? Helen , I know it's gross , but I ' Ve seen many public

In [11]:
inputs = tokenizer(["How are you?"], return_tensors="pt").to('cuda:0')
reply_ids = model_copy.generate(**inputs)
print("Bot: ", tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0])

/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/transformers/generation/utils.py:1350: UserWarning: Using `max_length`'s default (60) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Bot:   I'm doing well, thank you. How about yourself? Do you have any plans for the weekend?


In [12]:
env = MyRLEnv(
    model, tokenizer, 
    observation_input=observation_list, 
    max_length=75, 
    compare_sample=1
)
actor = TextRLActor(
    env, model, tokenizer, 
    act_deterministically=False, 
    optimizer='adamw',
    temperature=0.8,
    top_k=100,
    top_p=0.85
)
agent = actor.agent_ppo(
    update_interval=2, 
    minibatch_size=100, 
    epochs=10,
    lr=4e-4
)

In [13]:
train_agent_with_evaluation(
    agent,
    env,
    steps=1500,
    eval_n_steps=None,
    eval_n_episodes=3,
    eval_interval=180,
    outdir='bloom—test-9',
)

/mnt/hdd1/venkat/RL/Text_Generation/venv/lib/python3.7/site-packages/pfrl/agents/ppo.py:133: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  actions = torch.tensor([b["action"] for b in dataset], device=device)


You did what ? Helen , I know it's gross , but I ' Ve seen many public washrooms that are much worse . Why are you cleaning the counter top ? are you out of your mind ?
No, I am not out of my mind. I am just so disgusted by it.
I know. I feel like I did something really bad. I am so ashamed of myself. I think I am going to go back and wash it again
outdir:bloom—test-9 step:34 episode:0 R:-10.127623535925522
statistics:[('average_value', -0.89705276), ('average_entropy', 0.98550844), ('average_value_loss', 6.423647299052819), ('average_policy_loss', 9.88280075292176e-07), ('n_updates', 17), ('explained_variance', 0.7646925769125189)]
What ’ s wrong ? Didn ’ t you think it was fun ? !
I thought it was going to be a lot of fun, but I ended up not going.
I'm not sure, but it felt really bad.
outdir:bloom—test-9 step:46 episode:1 R:9.382722895301413
statistics:[('average_value', -0.86896044), ('average_entropy', 1.0257944), ('average_value_loss', 9.193812599061461), ('average_policy_loss', 

(<textrl.actor.TextPPO at 0x7f1d3048db00>,
 [{'average_value': 4.017539,
   'average_entropy': 1.1614664,
   'average_value_loss': 113.82944666219234,
   'average_policy_loss': -8.060598592419654e-07,
   'n_updates': 93,
   'explained_variance': 0.7555458677023109,
   'eval_score': 55.3703718802426},
  {'average_value': 3.0791447,
   'average_entropy': 1.0488197,
   'average_value_loss': 274.6607828978077,
   'average_policy_loss': 2.944588666409231e-07,
   'n_updates': 195,
   'explained_variance': -2.5676882808678005,
   'eval_score': 21.608223476291943},
  {'average_value': 3.4729586,
   'average_entropy': 0.9961737,
   'average_value_loss': 406.3529925113916,
   'average_policy_loss': 2.84093611853109e-07,
   'n_updates': 274,
   'explained_variance': -21.160546873967597,
   'eval_score': 14.996203693270218},
  {'average_value': 4.608681,
   'average_entropy': 0.8328681,
   'average_value_loss': 314.98279801871627,
   'average_policy_loss': 3.019988510999383e-07,
   'n_updates': 39

In [14]:
agent.load('./bloom—test-9/best')
test_string = 'How are you?'
# print(test_string)
actor.predict({'input': test_string})

How are you?
I'm doing well, thank you. How about yourself? Do you have any plans for the weekend?
I'm doing well, thanks for asking.


[" I'm doing well, thanks for asking.</s>"]

In [15]:
agent.load('./bloom—test-9/best')
for i in observation_list:
    s = actor.predict(i)[0]
    if get_emotion_score(s.replace('</s>', '')) > 0.6:
        print(i['input'])
        print(s)

So Dick , how about getting some coffee for tonight ?
That's a good idea. I think I'll do that. I need some caffeine.
I can't because I have a soccer game, but I can try.
Isn ’ t he the best instructor ? I think he ’ s so hot . Wow ! I really feel energized , don ’ t you ?
Yes, he is the best. I am so glad I got to see him in person.
I have been studying and practicing every day.  I am getting better and better.
What ’ s wrong ? Didn ’ t you think it was fun ? !
I thought it was going to be a lot of fun, but I ended up not going.
I am thinking it was a little fun, but I feel bad because I can't keep up.
What ’ s wrong ? Didn ’ t you think it was fun ? !
 I am thinking it was a little fun, but I feel bad because I can't keep up.</s>
What ?
I think it's time for me to go to bed. I've been up all night.
I am going to take some time to study for my exams.
Cats are beautiful and lovely , aren ’ t they ?
Yes, they are. I have two of them. They are so sweet and playful.
Yes, they are so lovin

KeyboardInterrupt: 